In [32]:
## https://colab.research.google.com/drive/1QpvUHQzpHPvlMgBElwd5NJQ6qpryVeWE?usp=sharing#scrollTo=OBh-7k6jsX7h
## https://colab.research.google.com/drive/1AAyEdTz-Z6ShKvewbt1ZHUICqak0MiwR?usp=sharing#scrollTo=qMSSWy9JTTuO (seoncd agent)


#todo: wieso geht snomet cd frage ned? wie csv kombinieren? https://www.reddit.com/r/LangChain/comments/13feb7l/langchain_csv_agent_with_other_llms/

import os
import constants

# get api key
os.environ["OPENAI_API_KEY"] = constants.APIKEY
os.environ["WOLFRAM_ALPHA_APPID"] = constants.WOLFRAM

### Creating an Agent

In [33]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory

In [24]:
llm = OpenAI(temperature=0.5)

### Loading Tools

In [50]:
tools = load_tools([ "wikipedia", "wolfram-alpha", "llm-math"], llm=llm)

In [29]:
tools = load_tools([], llm=llm)

In [51]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [52]:
# What is a tool like this
tools[0].name, tools[0].description

('Wikipedia',
 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.')

### Initializing the Agent

In [54]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)


In [37]:
agent = initialize_agent(tools, 
                        llm, 
                        agent="conversational-react-description", 
                        memory=memory, 
                        verbose=True)

In [8]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nwolfram_alpha: A wrapper around Wolfram Alpha. Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [wolfram_alpha]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [38]:
## Standard LLM Query
#agent.run("Hi How are you today?")
agent.run("Hey, who are you?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi there! I'm Assistant, an AI-powered language model. I'm here to help you with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

> Finished chain.


"Hi there! I'm Assistant, an AI-powered language model. I'm here to help you with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics."

In [39]:
agent.run("Who is the United States President? What is his current age raised divided by 2?")




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Joe Biden
Observation: Page: Joe Biden
Summary: Joseph Robinette Biden Jr. ( (listen) BY-dən; born November 20, 1942) is an American politician who is the 46th and current president of the United States. A member of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 under President Barack Obama and represented Delaware in the United States Senate from 1973 to 2009.
Born in Scranton, Pennsylvania, Biden moved with his family to Delaware in 1953. He studied at the University of Delaware before earning his law degree from Syracuse University. He was elected to the New Castle County Council in 1970 and to the U.S. Senate in 1972. As a senator, Biden drafted and led the effort to pass the Violent Crime Control and Law Enforcement Act and the Violence Against Women Act; and oversaw six U.S. Supreme Court confirmation hearings, including the conte

'The United States President is Joe Biden, and his current age raised divided by two is 78/2 = 39.'

In [55]:
agent.run("Is wolfram alpha better than gpt3?")



> Entering new AgentExecutor chain...
 I should compare the capabilities of wolfram alpha and gpt3
Action: Wikipedia
Action Input: GPT-3
Observation: Page: GPT-3
Summary: Generative Pre-trained Transformer 3 (GPT-3) is a large language model released by OpenAI in 2020. Like its predecessor GPT-2, it is a decoder-only transformer model of deep neural network, which uses attention in place of previous recurrence- and convolution-based architectures. Attention mechanisms allow the model to selectively focus on segments of input text it predicts to be the most relevant. It uses a 2048-tokens-long context and then-unprecedented size of 175 billion parameters, requiring 800GB to store. The model demonstrated strong zero-shot and few-shot learning on many tasks.Microsoft announced on September 22, 2020, that it had licensed "exclusive" use of GPT-3; others can still use the public API to receive output, but only Microsoft has access to GPT-3's underlying model.

Page: GPT-4
Summary: Generat

'Wolfram Alpha and GPT-3 both have powerful capabilities, but GPT-4 is the latest and most advanced model with added features such as the ability to take images as input.'

In [56]:
agent.run("What is the molar weight of creatinine and what is the conversion factor from umol/l to mg/dl?")



> Entering new AgentExecutor chain...
 I need to look up the molar weight and the conversion factor.
Action: wolfram_alpha
Action Input: molar weight of creatinine and conversion factor from umol/l to mg/dl
Observation: Wolfram Alpha wasn't able to answer it
Thought: I should look up the molar weight and do the conversion calculation
Action: Wikipedia
Action Input: creatinine
Observation: Page: Creatinine
Summary: Creatinine (; from Ancient Greek: κρέας (kréas) 'flesh') is a breakdown product of creatine phosphate from muscle and protein metabolism. It is released at a constant rate by the body (depending on muscle mass).

Page: Urea-to-creatinine ratio
Summary: In medicine, the urea-to-creatinine ratio (UCR), known in the United States as BUN-to-creatinine ratio, is the ratio of the blood levels of urea (BUN) (mmol/L) and creatinine (Cr) (μmol/L). BUN only reflects the nitrogen content of urea (MW 28) and urea measurement reflects the whole of the molecule (MW 60), urea is just over

'The molar weight of creatinine is 113 g/mol and the conversion factor from umol/l to mg/dl is 88.4.'

In [46]:
agent.run("I measured creatinine in plasma: the value was 190 umol/l. How much would that be in mg/dl? For this, first look up the molecular weight on wikipedia, then use wolfram alpha to convert it")



> Entering new AgentExecutor chain...
 I need to look up the molecular weight of creatinine and use wolfram alpha to convert the units
Action: Wikipedia
Action Input: Creatinine molecular weight
Observation: Page: Low-molecular-weight heparin
Summary: Low-molecular-weight heparin (LMWH) is a class of anticoagulant medications. They are used in the prevention of blood clots and treatment of venous thromboembolism (deep vein thrombosis and pulmonary embolism) and in the treatment of myocardial infarction.
Heparin is a naturally occurring polysaccharide that inhibits coagulation, the process that leads to thrombosis. Natural heparin consists of molecular chains of varying lengths, or molecular weights. Chains of varying molecular weights, from 5000 to over 40,000 Daltons, make up polydisperse pharmaceutical-grade heparin. LMWHs, in contrast, consist of only short chains of polysaccharide. LMWHs are defined as heparin salts having an average molecular weight of less than 8000 Da and for 

'190 umol/l of creatinine is equivalent to 2.1493 mg/dl.'

## New Agent

In [15]:
from langchain.csv_agents import CsvAgent
tools = load_tools([ "wikipedia", "wolfram-alpha", "csv"], llm=llm)
#"llm-math","terminal",

ModuleNotFoundError: No module named 'langchain.csv_agents'

In [20]:
from langchain import csv_agent

ImportError: cannot import name 'csv_agent' from 'langchain' (/Users/fabian/.virtualenvs/datastuff/lib/python3.9/site-packages/langchain/__init__.py)

In [14]:
agent = initialize_agent(tools, llm, agent="csv-agent", verbose=True)




In [ ]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, historical events, or other subjects. Input should be a search query.\nTerminal: Executes commands in a terminal. Input should be valid commands, and the output will be any output from running that command.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator, Wikipedia, Terminal]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now 

In [25]:
agent.run("What is the molar weight of creatinine and what is the conversion factor from umol/l to mg/dl?")




> Entering new AgentExecutor chain...
 I need to find out the molar weight of creatinine and the conversion factor from umol/l to mg/dl.
Action: Wolfram_alpha
Action Input: Molar weight of creatinine and conversion factor from umol/l to mg/dl
Observation: Wolfram_alpha is not a valid tool, try another one.
Thought: I need to find out the molar weight of creatinine and the conversion factor from umol/l to mg/dl.
Action: Wikipedia
Action Input: Creatinine
Observation: Page: Creatinine
Summary: Creatinine (; from Ancient Greek: κρέας (kréas) 'flesh') is a breakdown product of creatine phosphate from muscle and protein metabolism. It is released at a constant rate by the body (depending on muscle mass).

Page: Urea-to-creatinine ratio
Summary: In medicine, the urea-to-creatinine ratio (UCR), known in the United States as BUN-to-creatinine ratio, is the ratio of the blood levels of urea (BUN) (mmol/L) and creatinine (Cr) (μmol/L). BUN only reflects the nitrogen content of urea (MW 28) and

'The molar weight of creatinine is 113.12 g/mol and the conversion factor from umol/l to mg/dl is 88.4.'